In [161]:
import numpy as np
import pandas as pd

In [162]:
class Particle_of_swarm(object):
    def __init__(self, hiddenSize, inputSize, outputSize):
        # initiate layers
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize

        layers = [self.inputSize] + self.hiddenSize + [self.outputSize]

        # initiate positions
        positions = []
        positions_f = []
        for i in range(len(layers)-1):
            p = np.random.rand(layers[i], layers[i+1])
            positions_f = np.concatenate((positions_f,p.flatten()), axis=None)
            positions.append(p)
        self.positions = positions
        self.positions_f = positions_f

        velocitys = []
        for i in range(len(layers) - 1): 
            v = np.random.rand(layers[i+1])
            velocitys.append(v)
        self.velocitys = velocitys


        # for i in range(len(layers) - 1): 
        #     p_b = np.zeros(layers[i+1])
        #     p_best.append(p_b)
        # self.p_best = p_best

    def feedForward(self, X):
        Output_node = X
        for i, p in enumerate(self.positions):
            Output_node = np.dot(Output_node, p)

        return Output_node

    def object_funct(self,X,Y):
        
        sum_err = 0

        for j,input in enumerate(X):

            target = Y[j]
            output = self.feedForward(input)

            sum_err += self._mae(target,output)

        self.pbest = (1/(sum_err+1))
        return self.pbest

    def _mae(self, target, output):
        return np.average(abs(target - output))
        


In [163]:
# class Global_Best_Swarm(object):
#     def __init__(self, Praticle):
#         self.Praticle = Praticle

#     def feedForward(self, X):
#         output_node = X

#         for i, p in enumerate(self.Praticle.positions):

#             output_node = np.dot(output_node, p)

#         return output_node  

#     def objec_funct(self,X,Y,epoch):
#         self.feedForward(X)

#     def _mae(self, target, output):
#         return np.average(abs(target - output))

In [164]:
def _readfile(file):

    # ----- Clean NaN Values -----
    df = pd.read_csv(file)
    df = df.fillna(method = 'ffill')

    # ----- Create Features -----
    X = df[['PT08.S1(CO)','PT08.S2(NMHC)','PT08.S3(NOx)','PT08.S4(NO2)','PT08.S5(O3)','T','RH','AH']].copy(deep=False)
    X.drop(X.tail(240).index,inplace=True)

    # ----- Create Desired outputs -----
    label = df[['C6H6(GT)']].copy(deep=False)

    Y_10Day = label.iloc[240:,:].reset_index(drop=True)
    Y_10Day.rename(columns={"C6H6(GT)":"C6H6(GT)_10"}, inplace = True)

    Y_5Day = label.iloc[120:,:].reset_index(drop=True)
    Y_5Day.drop(Y_5Day.tail(120).index,inplace=True)
    Y_5Day.rename(columns={"C6H6(GT)":"C6H6(GT)_5"}, inplace = True)

    Y = pd.concat([Y_5Day,Y_10Day], axis=1)

    Input = X.to_numpy()
    Output = Y.to_numpy()
    return Input,Output

Input,Output = _readfile('data/AirQualityUCI.csv')


In [165]:
particles = []
p_bests = []
num_of_particle = 10

for i in range(0,num_of_particle):
    par = Particle_of_swarm([4],8,2)
    par.object_funct(Input,Output)
    p_bests.append(par.pbest)
    particles.append(par)

g_best = max(p_bests)

# for i in rang(0,num_of_particle):
#     if particles[i].pbest < p_bests[i]:
#         p_bests[i] = particles[i].pbest



